<a href="https://colab.research.google.com/github/vishnu190/Vishnu-chopra-INT248-CA1/blob/main/project_LANGUAGE_tRANSLATOR_CA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Importing library
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np
import tensorflow as tf

In [3]:
#Hyperparameters
batch_size = 64 #batch size for training data
latent_dim = 256  #Latent=dimensionality of the encoding space
num_samples = 10000  # number of samples to train on
epochs=100   # No: of epochs to train for

In [ ]:
# ********************************reading the data********************************
with open('/Language_dataset.txt', 'r', encoding='utf-8') as f: # f is the file 
  lines=f.read()

lines

In [6]:
#Vectorize the data.
input_texts = []
target_texts = []

input_chars = set()
target_chars = set()

with open('/Language_dataset.txt', 'r', encoding='utf-8') as f: # f is the file
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t') # making a list of strings
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_chars:
            input_chars.add(char)
    for char in target_text:
        if char not in target_chars:
            target_chars.add(char)

input_chars = sorted(list(input_chars)) # arranging in alphabetical order 
target_chars = sorted(list(target_chars))  # arranging in alphabetical order 
num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

#Print size
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)



Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [ ]:
print(input_chars)

In [ ]:
print(target_chars)

In [ ]:
print(input_texts)

In [ ]:
print(target_texts)

In [11]:
#Define data for encoder and decoder
# assigning every token indexes 
input_token_id = dict([(char, i) for i, char in enumerate(input_chars)])
target_token_id = dict([(char, i) for i, char in enumerate(target_chars)])
print(input_token_id)
print(target_token_id )

{' ': 0, '!': 1, '$': 2, '&': 3, "'": 4, ',': 5, '-': 6, '.': 7, '0': 8, '1': 9, '2': 10, '3': 11, '4': 12, '5': 13, '6': 14, '7': 15, '9': 16, ':': 17, '?': 18, 'A': 19, 'B': 20, 'C': 21, 'D': 22, 'E': 23, 'F': 24, 'G': 25, 'H': 26, 'I': 27, 'J': 28, 'K': 29, 'L': 30, 'M': 31, 'N': 32, 'O': 33, 'P': 34, 'Q': 35, 'R': 36, 'S': 37, 'T': 38, 'U': 39, 'V': 40, 'W': 41, 'Y': 42, 'Z': 43, 'a': 44, 'b': 45, 'c': 46, 'd': 47, 'e': 48, 'f': 49, 'g': 50, 'h': 51, 'i': 52, 'j': 53, 'k': 54, 'l': 55, 'm': 56, 'n': 57, 'o': 58, 'p': 59, 'q': 60, 'r': 61, 's': 62, 't': 63, 'u': 64, 'v': 65, 'w': 66, 'x': 67, 'y': 68, 'z': 69, '’': 70}
{'\t': 0, '\n': 1, ' ': 2, '!': 3, '$': 4, '&': 5, "'": 6, '(': 7, ')': 8, ',': 9, '-': 10, '.': 11, '0': 12, '1': 13, '5': 14, '6': 15, '9': 16, ':': 17, '?': 18, 'A': 19, 'B': 20, 'C': 21, 'D': 22, 'E': 23, 'F': 24, 'G': 25, 'H': 26, 'I': 27, 'J': 28, 'K': 29, 'L': 30, 'M': 31, 'N': 32, 'O': 33, 'P': 34, 'Q': 35, 'R': 36, 'S': 37, 'T': 38, 'U': 39, 'V': 40, 'Y': 41,

In [12]:
# performinf ONR HOT ENCODING  using numpy

encoder_in_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),dtype='float32')

decoder_in_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')


In [13]:

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_in_data[i, t, input_token_id[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_in_data[i, t, target_token_id[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_id[char]] = 1.

In [14]:
encoder_in_data[0].shape

(16, 71)

In [15]:
#Define and process the input sequence
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
#We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

#Using `encoder_states` set up the decoder as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [16]:
#Final model
# Define the model that will turn   "encoder_input_data" and "decoder_input_data"  into "decoder_target_data"
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [19]:
#Compiling and training the model
model.compile(tf.keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit([encoder_in_data, decoder_in_data], decoder_target_data,
          batch_size = batch_size,
          epochs=epochs, validation_split=0.2)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/100
125/125 [==============================] - 9s 44ms/step - loss: 0.4573 - accuracy: 0.1916 - val_loss: 0.6084 - val_accuracy: 0.2014
Epoch 2/100
125/125 [==============================] - 4s 34ms/step - loss: 0.4150 - accuracy: 0.2032 - val_loss: 0.5878 - val_accuracy: 0.2071
Epoch 3/100
125/125 [==============================] - 4s 34ms/step - loss: 0.4023 - accuracy: 0.2072 - val_loss: 0.5850 - val_accuracy: 0.2083
Epoch 4/100
125/125 [==============================] - 4s 34ms/step - loss: 0.3920 - accuracy: 0.2106 - val_loss: 0.5761 - val_accuracy: 0.2113
Epoch 5/100
125/125 [==============================] - 4s 34ms/step - loss: 0.3807 - accuracy: 0.2137 - val_loss: 0.5663 - val_accuracy: 0.2143
Epoch 6/100
125/125 [==============================] - 4s 34ms/step - loss: 0.3696 - accuracy: 0.2168 - val_loss: 0.5683 - val_accuracy: 0.2149
Epoch 7/100
125/125 [==============================] - 4s 34ms/step - loss: 0.3579 - accuracy: 0.2205 - val_loss: 0.5629 - val_accuracy:

In [20]:
#Model data Shape
print("encoder_in_data shape:",encoder_in_data.shape)
print("decoder_in_data shape:",decoder_in_data.shape)
print("decoder_target_data shape:",decoder_target_data.shape)


encoder_in_data shape: (10000, 16, 71)
decoder_in_data shape: (10000, 59, 93)
decoder_target_data shape: (10000, 59, 93)


In [22]:
#Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [23]:
# reverse-lookup token index to decode sequence back to something readable
reverse_input_char_index = dict((i, char) for char, i in input_token_id.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_id.items())

#Define Decode Sequence
def decode_sequence(input_seq):
    #Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    #Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    #Get the first character of target sequence with the start character.
    target_seq[0, 0, target_token_id['\t']] = 1.

    #Sampling loop for a batch of sequences
    #(to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        #Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        #Exit condition: either hit max length
        #or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        #Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        #Update states
        states_value = [h, c]

    return decoded_sentence

In [27]:
for seq_index in range(20): # we can increase the range to check all the outcomes , but i am keeping it 20 
  #take one sequense(part of the raining set)
  # for trying out decodong
    input_seq = encoder_in_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)


-
Input sentence: Go.
Decoded sentence: Sait.

-
Input sentence: Run!
Decoded sentence: Courez !

-
Input sentence: Run!
Decoded sentence: Courez !

-
Input sentence: Wow!
Decoded sentence: Ça alors.

-
Input sentence: Fire!
Decoded sentence: Attendrer.

-
Input sentence: Help!
Decoded sentence: wouvii.

-
Input sentence: Jump.
Decoded sentence: Sarte.

-
Input sentence: Stop!
Decoded sentence: Arrête-toi.

-
Input sentence: Stop!
Decoded sentence: Arrête-toi.

-
Input sentence: Stop!
Decoded sentence: Arrête-toi.

-
Input sentence: Wait!
Decoded sentence: Attends !

-
Input sentence: Wait!
Decoded sentence: Attends !

-
Input sentence: I see.
Decoded sentence: Je comprend.

-
Input sentence: I try.
Decoded sentence: J'essaye.

-
Input sentence: I won!
Decoded sentence: J'ai gagné !

-
Input sentence: I won!
Decoded sentence: J'ai gagné !

-
Input sentence: Oh no!
Decoded sentence: Je vous peue pas.

-
Input sentence: Attack!
Decoded sentence: Attaque !

-
Input sentence: Attack!
Decod